## Seaborn을 이용한 선형 회귀 및 데이터 시각화
- 이번 시간에는 영화 예산과 수익에 대한 자료를 들여다 본다. 이런 DataSet는 "Scikit-Learn"과 같은 툴을 통해 선형 회귀 분석을 실행하고, "Matplotlib"을 기반으로 구축된 데이터 시각화 라이브러리인 "Seaborn"을 사용하기에 매우 적합하다.

<img src="asset/movies.png">

- 이번 시간에 우리가 답할 질문은, 더 많은 영화 예산은 박스 오피스에서 더 많은 수익으로 이어지는지에 대한 것이다.
- 다른 말로, "영화 스튜디오는 더 많은 영화 제작을 위해 영화에 더 많은 돈을 써야하는 것인가?" 이다.

### 이번 시간 Study 내용 
1. "Seaborn"이라는, 유명한 데이터 시각화 라이브러리 사용 방법

<img src="asset/seaborn.png">

2. "Scikit-Learn"을 이용해 선형 회귀 분석을 실행하고 해석

<img src="asset/sklearn.png">

3. 데이터간의 관계를 시각화하는 산점도를 통해, 회귀 분석을 표시하는 방법

3. 산점도에 세 번째 차원을 추가하여 거품 차트를 만드는 방법

3. 내림 나눗셈(//)을 영리하게 사용해, 데이터를 변환하는 방법